# 0 Importing Packages

In [35]:
# Load the autoreload extension to automatically reload modules before executing code (to avoid restarting the kernel)
%load_ext autoreload 
# NB. uncomment the line above first time you run this cell
%autoreload 2

from pathlib import Path
import pandas as pd
import os
import networkx as nx

from resources.network_functions import GraphConstructor
from resources.generate_appendix_tables import generate_appendix

# Set max row view for pandas to 100
pd.set_option('display.max_rows', 100)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0.1 File Paths

In [3]:
fp_main = Path('/Volumes/SAM-SODAS-DISTRACT/Coding Distraction/github_as_a_market_device')
fp_main_output = Path(fp_main / 'output')   

# 1 Load in edgelist data

In [4]:
# Read in the data
all_edges_user_level = pd.read_parquet(fp_main_output / 'all_edges_user_level.gzip.parquet')
attention_edges_user_level = pd.read_parquet(fp_main_output / 'attention_edges_user_level.gzip.parquet')
collaboration_edges_user_level = pd.read_parquet(fp_main_output / 'collaboration_edges_user_level.gzip.parquet')

In [5]:
## 1.1 Construct the graphs
gc_attention = GraphConstructor(all_edges_user_level, graph_type='attention')
attention_graph = gc_attention.get_graph()
print("Attention Graph edges:", attention_graph.edges(data=True))

gc_collaboration = GraphConstructor(all_edges_user_level, graph_type='collaboration')
collaboration_graph = gc_collaboration.get_graph()
print("Collaboration Graph edges:", collaboration_graph.edges(data=True))

Attention Graph edges: [('charlie tango', 'charlie tango', {'weight': 11, 'follows': 12, 'stars': 14, 'watches': 127, 'd_intra_level': 1, 'd_inter_level': 0}), ('charlie tango', 'signifly', {'weight': 1, 'follows': 1, 'stars': 0, 'watches': 0, 'd_intra_level': 0, 'd_inter_level': 1}), ('knowit', 'miracle', {'weight': 1, 'follows': 0, 'stars': 0, 'watches': 6, 'd_intra_level': 0, 'd_inter_level': 1}), ('knowit', 'knowit', {'weight': 3, 'follows': 0, 'stars': 0, 'watches': 49, 'd_intra_level': 1, 'd_inter_level': 0}), ('trifork', 'trifork', {'weight': 119, 'follows': 40, 'stars': 107, 'watches': 1017, 'd_intra_level': 1, 'd_inter_level': 0}), ('kmd', 'kmd', {'weight': 4, 'follows': 0, 'stars': 0, 'watches': 9, 'd_intra_level': 1, 'd_inter_level': 0}), ('systematic', 'systematic', {'weight': 6, 'follows': 0, 'stars': 8, 'watches': 143, 'd_intra_level': 1, 'd_inter_level': 0}), ('oxygen', 'abtion', {'weight': 1, 'follows': 0, 'stars': 0, 'watches': 1, 'd_intra_level': 0, 'd_inter_level': 1

# 2 Descriptive statistics

## 2.1 Collaboration Network Stats

**General network statistics**

- no_users: Number of GitHub user-profiles in the graph
- no_companies: Number of companies in the graph
- no_users_edges: Number of unique user-to-user edges in the graph
- no_inter_company_edges_directed: Number of unique company-to-company edges in the graph
- no_inter_weight: Number of forks between users from different companies in the graph
- no_selfloops: Number of forks between users from the same company in the graph
- no_companies: Number of companies in the graph

In [53]:
# Nodes and edges
no_users = len(pd.unique(collaboration_edges_user_level[['src', 'target']].values.ravel()))
no_companies = len(set(collaboration_edges_user_level["src_company"]).union(collaboration_edges_user_level["target_company"]))
no_users_edge = int(sum(d.get("weight", 1) for _, _, d in collaboration_graph.edges(data=True)))
no_inter_company_edges_directed = len([(u,v) for u, v, d in collaboration_graph.edges(data=True) if d.get("d_inter_level") == 1])

# Total weight of inter-company edges (user-level, directed)
inter_company_mask = collaboration_edges_user_level['src_company'] != collaboration_edges_user_level['target_company']
no_inter_weight = collaboration_edges_user_level[inter_company_mask]['action'].value_counts().sum()

# Total weight of self-loop edges (src_company == tgt_company)
selfloop_mask = collaboration_edges_user_level['src_company'] == collaboration_edges_user_level['target_company']
no_selfloops = collaboration_edges_user_level[selfloop_mask]['action'].value_counts().sum()

def calculate_densities(directed_graph):
    # Create a copy of the graph to avoid modifying the original
    graph = directed_graph.copy()

    # Remove self-loops
    self_loops = list(nx.selfloop_edges(graph))
    graph.remove_edges_from(self_loops)

    # Calculate unweighted density
    number_of_nodes = graph.number_of_nodes()
    possible_edges = number_of_nodes * (number_of_nodes - 1)

    # Calculate weighted density
    sum_of_weights = sum(data['weight'] for u, v, data in graph.edges(data=True))
    weighted_density = sum_of_weights / possible_edges if possible_edges != 0 else 0

    return weighted_density

weighted_density_collaboration = calculate_densities(collaboration_graph)

print(f"No. users: {no_users}")
print(f"No. companies: {no_companies}")
print(f"User edges (directed): {no_users_edge}")
print(f"Inter-company edges (directed): {no_inter_company_edges_directed}")
print(f"Inter-company GH actions: {no_inter_weight}")
print(f"Intra-company GH actions: {no_selfloops}")
print(f"Weighted density (attention graph): {weighted_density_collaboration}")

No. users: 39
No. companies: 10
User edges (directed): 38
Inter-company edges (directed): 1
Inter-company GH actions: 1
Intra-company GH actions: 64
Weighted density (attention graph): 0.011111111111111112


**Describing GitHub user's actions on company-level (with self-loops/intra-company level)** 
- E.g. if *X*-user (working for company *Z*) has starred three repo's owned by *Y*-user (working for company *W*) this will give add three to the count of out-going edges for company *Z*, and three in-going edges for *W*. 

In [7]:
# Make tex

# Count outgoing and incoming edges and exclude self-loops
outgoing_counts = collaboration_edges_user_level.groupby('src_company').size().reset_index(name='outgoing_edges')
incoming_counts = collaboration_edges_user_level.groupby('target_company').size().reset_index(name='incoming_edges')

# Merge counts to get a single table
company_stats = pd.merge(outgoing_counts, incoming_counts, left_on='src_company', right_on='target_company', how='outer')

# Fill NaN values (in case some companies only have incoming or outgoing edges)
company_stats = company_stats.fillna(0)

# Rename columns for clarity
company_stats = company_stats[['src_company', 'outgoing_edges', 'incoming_edges']]
company_stats.columns = ['Company', 'Outgoing Edges', 'Incoming Edges']

# Sort by total edges
company_stats['Total Edges'] = company_stats['Outgoing Edges'] + company_stats['Incoming Edges']
company_stats = company_stats.sort_values('Total Edges', ascending=False)

# Convert to LaTeX table format
latex_table = company_stats.to_latex(index=False, caption="Company Network Edge Counts", label="tab:company_edges", column_format="lcc")

# Print the table
print(latex_table)
company_stats


\begin{table}
\caption{Company Network Edge Counts}
\label{tab:company_edges}
\begin{tabular}{lcc}
\toprule
Company & Outgoing Edges & Incoming Edges & Total Edges \\
\midrule
trifork & 37 & 38.000000 & 75.000000 \\
abtion & 6 & 6.000000 & 12.000000 \\
netcompany & 6 & 6.000000 & 12.000000 \\
signifly & 5 & 5.000000 & 10.000000 \\
eg a s & 3 & 3.000000 & 6.000000 \\
must & 3 & 3.000000 & 6.000000 \\
shape & 2 & 2.000000 & 4.000000 \\
knowit & 1 & 1.000000 & 2.000000 \\
uptime & 1 & 1.000000 & 2.000000 \\
miracle & 1 & 0.000000 & 1.000000 \\
\bottomrule
\end{tabular}
\end{table}



,Company,Outgoing Edges,Incoming Edges,Total Edges
8,trifork,37,38.0,75.0
0,abtion,6,6.0,12.0
5,netcompany,6,6.0,12.0
7,signifly,5,5.0,10.0
1,eg a s,3,3.0,6.0
4,must,3,3.0,6.0
6,shape,2,2.0,4.0
2,knowit,1,1.0,2.0
9,uptime,1,1.0,2.0
3,miracle,1,0.0,1.0


**Describing GitHub user's actions on company-level (only inter-company level)** 

In [8]:
# Step 1: Exclude self-loops
company_edges = collaboration_edges_user_level[collaboration_edges_user_level['src_company'] != collaboration_edges_user_level['target_company']]

# Step 2: Count outgoing and incoming edges
outgoing_counts = company_edges.groupby('src_company').size().reset_index(name='outgoing_edges')
incoming_counts = company_edges.groupby('target_company').size().reset_index(name='incoming_edges')

# Step 3: Get all unique companies from both columns
all_companies = pd.DataFrame(pd.unique(company_edges[['src_company', 'target_company']].values.ravel()), columns=['src_company'])

# Step 4: Merge outgoing and incoming counts separately to ensure all companies are included
company_stats = all_companies.merge(outgoing_counts, on='src_company', how='left').merge(
    incoming_counts, left_on='src_company', right_on='target_company', how='left'
)

# Step 5: Drop duplicate target_company column and fill NaN with 0
company_stats = company_stats[['src_company', 'outgoing_edges', 'incoming_edges']].fillna(0)

# Step 6: Convert to integer type
company_stats[['outgoing_edges', 'incoming_edges']] = company_stats[['outgoing_edges', 'incoming_edges']].astype(int)

# Step 7: Sort by total edges
company_stats['total_edges'] = company_stats['outgoing_edges'] + company_stats['incoming_edges']
company_stats = company_stats.sort_values('total_edges', ascending=False).reset_index(drop=True)

# Step 7: Convert to LaTeX table format
latex_table = company_stats.to_latex(index=False, caption="Company Network Edge Counts (Excluding Self-Loops)", 
                                     label="tab:collaboration", column_format="lcc")

# Print LaTeX table
print(latex_table)
company_stats

\begin{table}
\caption{Company Network Edge Counts (Excluding Self-Loops)}
\label{tab:collaboration}
\begin{tabular}{lcc}
\toprule
src_company & outgoing_edges & incoming_edges & total_edges \\
\midrule
miracle & 1 & 0 & 1 \\
trifork & 0 & 1 & 1 \\
\bottomrule
\end{tabular}
\end{table}



,src_company,outgoing_edges,incoming_edges,total_edges
0,miracle,1,0,1
1,trifork,0,1,1


In [32]:
# Step 1: Filter for edges where companies belong to different categories
filtered_edges = collaboration_edges_user_level[
    collaboration_edges_user_level['src_company'] != collaboration_edges_user_level['target_company']
]

# Step 1: Group by 'src_company_label', 'target_company_label' and 'action', and count occurrences
category_action_counts = (
    filtered_edges
    .groupby(['src_company_label', 'target_company_label', 'action'])
    .size()
    .reset_index(name='count')
)

# Step 2: Pivot the table to get counts of each action as columns
pivoted_df = category_action_counts.pivot_table(
    index=['src_company_label', 'target_company_label'],
    columns='action',
    values='count',
    aggfunc='sum',
    fill_value=0
).reset_index()
pivoted_df.rename(columns={'forks': 'no_of_forks'}, inplace=True)

pivoted_df

# Optional: Save to .tex file
#with open("collaboration_table.tex", "w") as f:
#    f.write(latex_table)

action,src_company_label,target_company_label,no_of_forks
0,2 Bespoke app companies,1 Digital and marketing consultancies,1


## 2.2 Attention Network Stats

**General network statistics**

- no_users: Number of GitHub user-profiles in the graph
- no_companies: Number of companies in the graph
- no_users_edges: Number of unique user-to-user edges in the graph
- no_inter_company_edges_directed: Number of unique company-to-company edges in the graph
- no_inter_weight: Number of all attention GH-actions between users from different companies in the graph
- no_selfloops: Number of all attention GH-actions between users from the same company in the graph
- no_companies: Number of companies in the graph

In [ ]:
# Nodes and edges
no_users = len(pd.unique(attention_edges_user_level[['src', 'target']].values.ravel()))
no_companies = len(set(attention_edges_user_level["src_company"]).union(attention_edges_user_level["target_company"]))
no_users_edge = int(sum(d.get("weight", 1) for _, _, d in attention_graph.edges(data=True)))
no_inter_company_edges_directed = len([(u,v) for u, v, d in attention_graph.edges(data=True) if d.get("d_inter_level") == 1])

# Total weight of inter-company edges (user-level, directed)
inter_company_mask = attention_edges_user_level['src_company'] != attention_edges_user_level['target_company']
no_inter_weight = attention_edges_user_level[inter_company_mask]['action'].value_counts().sum()

# Total weight of self-loop edges (src_company == tgt_company)
selfloop_mask = attention_edges_user_level['src_company'] == attention_edges_user_level['target_company']
no_selfloops = attention_edges_user_level[selfloop_mask]['action'].value_counts().sum()

# Density
weighted_density_attention = calculate_weighted_density(attention_graph)


print(f"No. users: {no_users}")
print(f"No. companies: {no_companies}")
print(f"User edges (directed): {no_users_edge}")
print(f"Inter-company edges (directed): {no_inter_company_edges_directed}")
print(f"Inter-company GH actions: {no_inter_weight}")
print(f"Intra-company GH actions: {no_selfloops}")
print(f"Weighted density (attention graph): {weighted_density_attention}")

No. users: 128
No. companies: 16
User edges (directed): 215
Inter-company edges (directed): 6
Inter-company GH actions: 12
Intra-company GH actions: 2244
Weighted density (attention graph): 0.025
Unweighted density (attention graph): 0.025


**Describing GitHub user's actions on company-level (with self-loops/intra-company level)** 
- E.g. if *X*-user (working for company *Z*) has starred three repo's owned by *Y*-user (working for company *W*) this will give add three to the count of out-going edges for company *Z*, and three in-going edges for *W*. 

In [11]:
# Count outgoing and incoming edges and exclude self-loops
outgoing_counts = attention_edges_user_level.groupby('src_company').size().reset_index(name='outgoing_edges')
incoming_counts = attention_edges_user_level.groupby('target_company').size().reset_index(name='incoming_edges')

# Merge counts to get a single table
company_stats = pd.merge(outgoing_counts, incoming_counts, left_on='src_company', right_on='target_company', how='outer')

# Fill NaN values (in case some companies only have incoming or outgoing edges)
company_stats = company_stats.fillna(0)

# Rename columns for clarity
company_stats = company_stats[['src_company', 'outgoing_edges', 'incoming_edges']]
company_stats.columns = ['Company', 'Outgoing Edges', 'Incoming Edges']

# Sort by total edges
company_stats['Total Edges'] = company_stats['Outgoing Edges'] + company_stats['Incoming Edges']
company_stats = company_stats.sort_values('Total Edges', ascending=False)

# Convert to LaTeX table format
latex_table = company_stats.to_latex(index=False, caption="Company Network Edge Counts", label="tab:company_edges", column_format="lcc")

# Print the table
print(latex_table)
company_stats


\begin{table}
\caption{Company Network Edge Counts}
\label{tab:company_edges}
\begin{tabular}{lcc}
\toprule
Company & Outgoing Edges & Incoming Edges & Total Edges \\
\midrule
trifork & 1164.000000 & 1165 & 2329.000000 \\
netcompany & 270.000000 & 267 & 537.000000 \\
charlie tango & 154.000000 & 153 & 307.000000 \\
systematic & 151.000000 & 151 & 302.000000 \\
shape & 131.000000 & 131 & 262.000000 \\
abtion & 111.000000 & 111 & 222.000000 \\
must & 108.000000 & 108 & 216.000000 \\
knowit & 55.000000 & 49 & 104.000000 \\
signifly & 44.000000 & 47 & 91.000000 \\
uptime & 36.000000 & 36 & 72.000000 \\
oxygen & 11.000000 & 11 & 22.000000 \\
kmd & 9.000000 & 9 & 18.000000 \\
capgemini & 8.000000 & 8 & 16.000000 \\
kruso & 3.000000 & 3 & 6.000000 \\
0 & 0.000000 & 6 & 6.000000 \\
fellowmind & 1.000000 & 1 & 2.000000 \\
\bottomrule
\end{tabular}
\end{table}



,Company,Outgoing Edges,Incoming Edges,Total Edges
13,trifork,1164.0,1165,2329.0
8,netcompany,270.0,267,537.0
2,charlie tango,154.0,153,307.0
12,systematic,151.0,151,302.0
10,shape,131.0,131,262.0
0,abtion,111.0,111,222.0
7,must,108.0,108,216.0
5,knowit,55.0,49,104.0
11,signifly,44.0,47,91.0
14,uptime,36.0,36,72.0


**Describing GitHub user's actions on company-level (only inter-company level)** 

In [12]:
# Step 1: Exclude self-loops
filtered_edges = attention_edges_user_level[attention_edges_user_level['src_company'] != attention_edges_user_level['target_company']]

# Step 2: Count outgoing and incoming edges
outgoing_counts = filtered_edges.groupby('src_company').size().reset_index(name='outgoing_edges')
incoming_counts = filtered_edges.groupby('target_company').size().reset_index(name='incoming_edges')

# Step 3: Get all unique companies from both columns
all_companies = pd.DataFrame(pd.unique(filtered_edges[['src_company', 'target_company']].values.ravel()), columns=['src_company'])

# Step 4: Merge outgoing and incoming counts separately to ensure all companies are included
company_stats = all_companies.merge(outgoing_counts, on='src_company', how='left').merge(
    incoming_counts, left_on='src_company', right_on='target_company', how='left'
)

# Step 5: Drop duplicate target_company column and fill NaN with 0
company_stats = company_stats[['src_company', 'outgoing_edges', 'incoming_edges']].fillna(0)

# Step 6: Convert to integer type
company_stats[['outgoing_edges', 'incoming_edges']] = company_stats[['outgoing_edges', 'incoming_edges']].astype(int)

# Step 7: Sort by total edges
company_stats['total_edges'] = company_stats['outgoing_edges'] + company_stats['incoming_edges']
company_stats = company_stats.sort_values('total_edges', ascending=False).reset_index(drop=True)

# Step 7: Convert to LaTeX table format
latex_table = company_stats.to_latex(index=False, caption="Company Network Edge Counts (Excluding Self-Loops)", 
                                     label="tab:collaboration", column_format="lcc")

# Print LaTeX table
print(latex_table)
company_stats

\begin{table}
\caption{Company Network Edge Counts (Excluding Self-Loops)}
\label{tab:collaboration}
\begin{tabular}{lcc}
\toprule
src_company & outgoing_edges & incoming_edges & total_edges \\
\midrule
knowit & 6 & 0 & 6 \\
miracle & 0 & 6 & 6 \\
netcompany & 3 & 0 & 3 \\
signifly & 0 & 3 & 3 \\
oxygen & 1 & 1 & 2 \\
abtion & 1 & 1 & 2 \\
charlie tango & 1 & 0 & 1 \\
trifork & 0 & 1 & 1 \\
\bottomrule
\end{tabular}
\end{table}



,src_company,outgoing_edges,incoming_edges,total_edges
0,knowit,6,0,6
1,miracle,0,6,6
2,netcompany,3,0,3
3,signifly,0,3,3
4,oxygen,1,1,2
5,abtion,1,1,2
6,charlie tango,1,0,1
7,trifork,0,1,1


**Describing actions based on company categories**

In [29]:
# Step 1: Filter for edges where companies belong to different categories
filtered_edges = attention_edges_user_level[
    attention_edges_user_level['src_company'] != attention_edges_user_level['target_company']
]

# Step 2: Group by 'src_company_label', 'target_company_label' and 'action', and count occurrences
category_action_counts = (
    filtered_edges
    .groupby(['src_company_label', 'target_company_label', 'action'])
    .size()
    .reset_index(name='count')
)

# Step 3: Pivot the table to get counts of each action as columns
pivoted_df = category_action_counts.pivot_table(
    index=['src_company_label', 'target_company_label'],
    columns='action',
    values='count',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Step 4: Sum the attention actions into a single column
pivoted_df['no_attention_actions'] = (
    pivoted_df.get('follows', 0) +
    pivoted_df.get('stars', 0) +
    pivoted_df.get('watches', 0)
)

# Generate and print LaTeX table
latex_table = pivoted_df.to_latex(index=False)
print(latex_table)

# Print table without index
pivoted_df

# Optional: Save to .tex file
#with open("attention_table.tex", "w") as f:
#    f.write(latex_table)

\begin{tabular}{llrrrr}
\toprule
src_company_label & target_company_label & follows & stars & watches & no_attention_actions \\
\midrule
1 Digital and marketing consultancies & 1 Digital and marketing consultancies & 1 & 3 & 2 & 6 \\
1 Digital and marketing consultancies & 2 Bespoke app companies & 0 & 0 & 6 & 6 \\
\bottomrule
\end{tabular}



action,src_company_label,target_company_label,follows,stars,watches,no_attention_actions
0,1 Digital and marketing consultancies,1 Digital and marketing consultancies,1,3,2,6
1,1 Digital and marketing consultancies,2 Bespoke app companies,0,0,6,6


# 3.0 Generate appendix tables for collaboration and attention edges in appendix.md

In [16]:
# Create output folder path
appendix_dir = "../appendix"
os.makedirs(appendix_dir, exist_ok=True)

# Collaboration Appendix
generate_appendix(
    df=collaboration_edges_user_level,
    src_col='src_company_label',
    target_col='target_company_label',
    action_col='action',
    new_column_name='no_collaboration_actions',
    relevant_actions=['forks'],
    filename="appendix_a_collaboration_inter_company_category_summary.md",
    appendix_label="Appendix A",
    appendix_title="Collaboration Between Companies of Different Categories",
    appendix_dir=appendix_dir
)

# Attention Appendix
generate_appendix(
    df=attention_edges_user_level,
    src_col='src_company_label',
    target_col='target_company_label',
    action_col='action',
    new_column_name='no_attention_actions',
    relevant_actions=['follows', 'stars', 'watches'],
    filename="appendix_b_attention_inter_company_category_summary.md",
    appendix_label="Appendix B",
    appendix_title="Attention Between Companies of Different Categories",
    appendix_dir=appendix_dir
)